In [1]:
import pandas as pd
import numpy as np
from keras.layers import Embedding, Flatten, dot,Input,Dropout,Dense
from keras.models import Model


from keras.models import Sequential
movies=pd.read_csv('ml-latest-small/movies.csv')
links=pd.read_csv('ml-latest-small/links.csv')
# ratings=pd.read_csv('ml-latest/ratings.csv')
tags=pd.read_csv('ml-latest-small/tags.csv')
ratings=pd.read_csv('rat.csv')




Using TensorFlow backend.


In [2]:
ratings['userid'] = ratings.userId.astype('category').cat.codes.values
ratings['movieid'] = ratings.movieId.astype('category').cat.codes.values

In [3]:
n_users, n_movies = len(ratings.userId.unique()), len(ratings.movieId.unique())
ratings


,userId,movieId,rating,timestamp,userid,movieid
0,1,2,3.5,1112486027,0,1
1,1,29,3.5,1112484676,0,28
2,1,32,3.5,1112484819,0,31
3,1,47,3.5,1112484727,0,46
4,1,50,3.5,1112484580,0,49
...,...,...,...,...,...,...
1048570,7120,168,5.0,1175543061,7119,163
1048571,7120,253,4.0,1175542225,7119,247
1048572,7120,260,5.0,1175542035,7119,254
1048573,7120,261,4.0,1175543376,7119,255


In [32]:
# model=cfmodel(max_userid,max_movieid,100)
# model.compile(loss='mse', optimizer='adamax')
from sklearn.model_selection import train_test_split
train, test = train_test_split(ratings, test_size=0.2)


In [25]:
k_factors=15
m = Input(shape=(1,))
u = Input(shape=(1,))
M=Embedding(n_movies,k_factors)(m)
m_vec=Flatten()(M)
m_vec=Dropout(0.4)(m_vec)

U=Embedding(n_users,k_factors)(u)
u_vec=Flatten()(U)
u_vec=Dropout(0.4)(u_vec)

dot_product = dot([m_vec,u_vec],axes=1)
out=Dense(1,activation='relu')(dot_product)



model = Model(inputs=[m,u], outputs=out)
model.compile(optimizer="adam",loss="mean_squared_error")
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1, 15)        420780      input_3[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 15)        213600      input_4[0][0]                    
____________________________________________________________________________________________

In [61]:
model.fit([ratings.movieid,ratings.userid],ratings.rating,validation_split=0.2,epochs=15,batch_size=150)

Train on 838860 samples, validate on 209715 samples
Epoch 1/15
838860/838860 [==============================] - 53s 63us/step - loss: 0.7825 - val_loss: 0.7073
Epoch 2/15
838860/838860 [==============================] - 52s 61us/step - loss: 0.7758 - val_loss: 0.7158
Epoch 3/15
838860/838860 [==============================] - 49s 58us/step - loss: 0.7711 - val_loss: 0.7207
Epoch 4/15
838860/838860 [==============================] - 50s 59us/step - loss: 0.7667 - val_loss: 0.7271
Epoch 5/15
838860/838860 [==============================] - 50s 59us/step - loss: 0.7649 - val_loss: 0.7376
Epoch 6/15
838860/838860 [==============================] - 51s 60us/step - loss: 0.7612 - val_loss: 0.7420
Epoch 7/15
838860/838860 [==============================] - 52s 62us/step - loss: 0.7594 - val_loss: 0.7459
Epoch 8/15
838860/838860 [==============================] - 52s 62us/step - loss: 0.7584 - val_loss: 0.7521
Epoch 9/15
838860/838860 [==============================] - 52s 62us/step - loss: 0.

In [55]:
model.predict([np.array([2540,]),np.array([1411,])])

array([[2.7125006]], dtype=float32)

In [62]:
y_hat = np.round(model.predict([test.userid, test.movieid]),0)
y_true = test.rating

In [63]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_true, y_hat)

0.9478172758267173

In [47]:
test[test.userid==1411]

,userId,movieId,rating,timestamp,userid,movieid
205548,1412,1125,3.0,945762178,1411,1077
205535,1412,923,4.0,946234775,1411,885
205644,1412,2685,3.0,983499705,1411,2531
205698,1412,5060,4.0,945761808,1411,4804
205620,1412,2249,3.0,970021201,1411,2105
205601,1412,1784,4.0,945762075,1411,1664
205668,1412,2959,4.0,945761572,1411,2796
205607,1412,1914,4.0,945761938,1411,1774
205565,1412,1256,4.0,979255791,1411,1199
205674,1412,3083,4.0,975907070,1411,2915


In [44]:
train[train.userid==1411]

,userId,movieId,rating,timestamp,userid,movieid
205579,1412,1307,2.0,945762133,1411,1249
205512,1412,514,2.0,945762297,1411,504
205631,1412,2395,4.0,968866208,1411,2249
205494,1412,162,5.0,945761706,1411,157
205504,1412,440,2.0,945762075,1411,430
...,...,...,...,...,...,...
205613,1412,2019,4.0,949522142,1411,1879
205568,1412,1265,3.0,945761865,1411,1208
205652,1412,2716,4.0,945762524,1411,2560
205543,1412,1077,2.0,979255931,1411,1032


In [60]:
 len(train.movieId.unique())

13458

In [59]:
 len(ratings.movieId.unique())

14026